In [20]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = 'max_split_size_mb:512'
# Load the data set
file_loc_train = 'sample_data/train.csv'
file_loc_test = 'sample_data/test.csv'
df = pd.read_csv(file_loc_train)
df_test = pd.read_csv(file_loc_test)

# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [21]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_data(df, tokenizer, max_length=128):
    return tokenizer(
        df['text'].tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = tokenize_data(train_df, tokenizer)
val_encodings = tokenize_data(val_df, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
import torch
from torch.utils.data import Dataset

class DisasterDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Convert labels to a tensor
train_labels = torch.tensor(train_df['target'].tolist())
val_labels = torch.tensor(val_df['target'].tolist())

# Create datasets
train_dataset = DisasterDataset(train_encodings, train_labels)
val_dataset = DisasterDataset(val_encodings, val_labels)

In [23]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-943f00ac1934>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-943f00ac1934>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss


In [ ]:
eval_results = trainer.evaluate()
#print(f"Validation Accuracy: {eval_results['eval_accuracy']}")
eval_results

In [ ]:
test = list(df_test['text'])
test_encodings = tokenizer(test, padding=True, truncation=True, max_length=128, return_tensors='pt')

# Model prediction
outputs = model(**test_encodings)
predictions = torch.argmax(outputs.logits, dim=1)

# Output prediction results
print(predictions)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.is_available()

In [ ]:
# Check if GPU is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Move the model to the appropriate device
model.to(device)

# Tokenize the test data
test = list(df_test['text'])
test_encodings = tokenizer(test, padding=True, truncation=True, max_length=128, return_tensors='pt')

# Move the input tensors to the same device as the model
test_encodings = {key: val.to(device) for key, val in test_encodings.items()}

# Model prediction
outputs = model(**test_encodings)

# Get the predicted classes
predictions = torch.argmax(outputs.logits, dim=1)

# Output prediction results
print(predictions)

In [ ]:
# Check if GPU is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Move the model to the appropriate device
model.to(device)

# Tokenize the test data in batches to reduce memory usage
batch_size = 16 # Experiment with different batch sizes
predictions = []
for i in range(0, len(df_test['text']), batch_size):
    test_batch = list(df_test['text'][i:i+batch_size])
    test_encodings = tokenizer(test_batch, padding=True, truncation=True, max_length=128, return_tensors='pt')

    # Move the input tensors to the same device as the model
    test_encodings = {key: val.to(device) for key, val in test_encodings.items()}

    # Model prediction
    with torch.no_grad(): # Use torch.no_grad() to reduce memory usage during inference
        outputs = model(**test_encodings)

    # Get the predicted classes
    batch_predictions = torch.argmax(outputs.logits, dim=1)
    predictions.extend(batch_predictions.cpu().numpy()) # Move predictions to CPU and convert to numpy array

# Output prediction results
print(predictions)

In [ ]:
"""with batch = 8 eval_loss': 0.6313741207122803
   with batch = 16 eval_loss': 0.5087562203407288"""